In [1]:
from transformers import GPT2LMHeadModel

/home/htkumar/local/miniconda3/envs/gpt-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from datasets import load_dataset

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.is_bf16_supported()

True